In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1. Carregar o dataframe detalhado
# Certifique-se de que este arquivo está no mesmo diretório do seu notebook,
# ou forneça o caminho completo para ele.
file_path = "audiencia_espelho_merged_segundos.csv"
df = None # Inicializa df como None para checagem posterior

try:
    df = pd.read_csv(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{file_path}' não encontrado.")
    print("Por favor, verifique o nome e o local do arquivo.")
    # Em um notebook, o fluxo continuará, mas df será None.

# Prosseguir somente se o dataframe foi carregado com sucesso
if df is not None:
    # 2. Pré-processamento inicial
    df['datetime_segundo_audiencia'] = pd.to_datetime(df['datetime_segundo_audiencia'], errors='coerce')
    df = df.sort_values('datetime_segundo_audiencia')

    # 3. Preparar dados focando em retrancas de conteúdo
    df_filtrado = df[df['Retranca'].notna() & (df['Retranca'] != "Não disponível")].copy()
    
    tipos_a_excluir = ['ABER', 'IMG', 'VINH']
    tipos_excluidos_str = ", ".join(tipos_a_excluir)
    
    tipo_col_name = None
    if 'tipo' in df_filtrado.columns:
        tipo_col_name = 'tipo'
    elif 'Tipo' in df_filtrado.columns:
        tipo_col_name = 'Tipo'
        
    if tipo_col_name:
        print(f"Coluna '{tipo_col_name}' encontrada. Excluindo tipos: {tipos_excluidos_str}.")
        df_retrancas_validas = df_filtrado[~df_filtrado[tipo_col_name].isin(tipos_a_excluir)].copy()
    else:
        print(f"AVISO: Nenhuma coluna 'tipo' ou 'Tipo' foi encontrada. O filtro por tipo de retranca não foi aplicado.")
        df_retrancas_validas = df_filtrado.copy()


    if df_retrancas_validas.empty:
        print("Não há dados de 'Retranca' definidos para esta análise após os filtros.")
    elif 'GLOBO' not in df_retrancas_validas.columns or not pd.api.types.is_numeric_dtype(df_retrancas_validas['GLOBO']):
        print("ERRO: Coluna 'GLOBO' não encontrada ou não é numérica.")
    else:
        print("Identificando instâncias de Retrancas e calculando métricas...")
        
        # 4. Identificar instâncias (blocos) de Retrancas
        df_retrancas_validas['Retranca_Shifted'] = df_retrancas_validas['Retranca'].shift(1)
        df_retrancas_validas['novo_bloco_retranca'] = (df_retrancas_validas['Retranca'] != df_retrancas_validas['Retranca_Shifted'])
        df_retrancas_validas['bloco_id_sequencial'] = df_retrancas_validas['novo_bloco_retranca'].cumsum()

        # 5. Calcular métricas por instância de Retranca
        inicio_blocos_retranca = df_retrancas_validas.groupby('bloco_id_sequencial').agg(Retranca=('Retranca', 'first'),horario_inicio_bloco=('datetime_segundo_audiencia', 'min')).reset_index()
        audiencia_por_bloco = df_retrancas_validas.groupby('bloco_id_sequencial').agg(audiencia_media_bloco=('GLOBO', 'mean'),duracao_bloco_segundos=('GLOBO', 'count')).reset_index()
        df_plot_temporal_barras = pd.merge(inicio_blocos_retranca, audiencia_por_bloco, on='bloco_id_sequencial')
        
        df_plot_temporal_barras = df_plot_temporal_barras.sort_values(by='horario_inicio_bloco')
        
        # 7. Criar label para o eixo X
        df_plot_temporal_barras['Retranca'] = df_plot_temporal_barras['Retranca'].astype(str)
        
        # --- Lógica para criar labels únicos e evitar sobreposição ---
        df_plot_temporal_barras['ocorrencia'] = df_plot_temporal_barras.groupby('Retranca').cumcount() + 1
        df_plot_temporal_barras['total_ocorrencias'] = df_plot_temporal_barras.groupby('Retranca')['Retranca'].transform('count')
        df_plot_temporal_barras['label_eixo_x'] = df_plot_temporal_barras.apply(
            lambda row: f"{row['Retranca']} ({row['ocorrencia']})" if row['total_ocorrencias'] > 1 else row['Retranca'],
            axis=1
        )


        if df_plot_temporal_barras.empty:
            print(f"Nenhuma instância de retranca encontrada após os filtros.")
        else:
            num_instancias = len(df_plot_temporal_barras)
            print(f"Preparando gráfico de BARRAS para {num_instancias} instâncias de Retrancas em ordem de aparição...")

            media_programa = 8
            df_plot_temporal_barras['Performance'] = np.where(
                df_plot_temporal_barras['audiencia_media_bloco'] > media_programa,
                'Acima da média',
                'Abaixo ou na média'
            )

            chart_width = max(800, num_instancias * 35)

            # 8. Criar o Gráfico de Barras Verticais
            fig = px.bar(
                df_plot_temporal_barras,
                x='label_eixo_x',
                y='audiencia_media_bloco',
                title='Audiência por Matérias',
                labels={'label_eixo_x': 'Matérias (Ordem de Aparição)', 
                        'audiencia_media_bloco': 'Audiência Domiciliar (pp)',
                        'Performance': 'Legenda'},
                hover_data={'Retranca': True, 'horario_inicio_bloco': True, 'duracao_bloco_segundos': True,'label_eixo_x': False },
                color='Performance',
                color_discrete_map={
                    'Acima da média': 'mediumseagreen',
                    'Abaixo ou na média': 'cornflowerblue'
                },
                width=chart_width
            )
            
            fig.update_traces(
                hovertemplate="<b>Matéria:</b> %{customdata[0]}<br>" + "<b>Início:</b> %{customdata[1]|%H:%M:%S}<br>" + "<b>Audiência Média:</b> %{y:.2f}<br>" + "<b>Duração:</b> %{customdata[2]}s<extra></extra>"
            )
            
            fig.update_traces(
                texttemplate='%{y:.2f}',
                textposition='inside',
                insidetextanchor='middle',
                textfont=dict(
                    color='white',
                    size=12
                ),
                constraintext='inside'
            )

            # 9. Estilizar o Gráfico
            nome_programa = "BDES" 
            
            # ALTERAÇÃO: A data da análise agora é definida manualmente aqui.
            # Basta editar o texto dentro das aspas.
            data_analise = "13/06/2025"
            
            fig.update_layout(
                xaxis_title='Matéria em Ordem de Aparição no Programa',
                yaxis_title='Audiência Domiciliar (pp)',
                title={
                    'text': f"<b>Audiência por Matérias - {nome_programa}</b><br>" +
                            f"<sup>Análise de: {data_analise}  |  Fonte: Kantar - Ibope</sup><br>" +
                            f"<sup><i>Desconsideradas retrancas do tipo: {tipos_excluidos_str}</i></sup>",
                    'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                    'font': {'size': 18}
                },
                xaxis_tickangle=-60,
                showlegend=True
            )
            
            fig.add_hline(
                y=8.19,
                line_width=2, 
                line_dash="dot", 
                line_color="black"
            )
            
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='lines',
                line=dict(color='black', dash='dot', width=2),
                name='Média simples do dia (8.19)',
                showlegend=True
            ))

            fig.update_xaxes(categoryorder='array', categoryarray=df_plot_temporal_barras['label_eixo_x'])

            # 10. Salvar em HTML
            output_html_path_temporal_barras = "audiencia_por_materias_BDES.html"
            fig.write_html(output_html_path_temporal_barras)

            print(f"\nGráfico de Barras de Audiência de Retrancas salvo como: '{output_html_path_temporal_barras}'")
            print("Abra este arquivo HTML em um navegador para interagir com o gráfico.")
            
            # Para exibir diretamente no Jupyter Notebook, descomente a linha abaixo:
            # fig.show()
else:
    print("\nO DataFrame não foi carregado. A visualização não pode ser gerada.")

Arquivo 'audiencia_espelho_merged_segundos.csv' carregado com sucesso!
Coluna 'Tipo' encontrada. Excluindo tipos: ABER, IMG, VINH.
Identificando instâncias de Retrancas e calculando métricas...
Preparando gráfico de BARRAS para 58 instâncias de Retrancas em ordem de aparição...

Gráfico de Barras de Audiência de Retrancas salvo como: 'audiencia_por_materias_BDES.html'
Abra este arquivo HTML em um navegador para interagir com o gráfico.


In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1. Carregar o dataframe detalhado
# Certifique-se de que este arquivo está no mesmo diretório do seu notebook,
# ou forneça o caminho completo para ele.
file_path = "audiencia_espelho_merged_segundos.csv"
df = None # Inicializa df como None para checagem posterior

try:
    df = pd.read_csv(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{file_path}' não encontrado.")
    print("Por favor, verifique o nome e o local do arquivo.")

# Prosseguir somente se o dataframe foi carregado com sucesso
if df is not None:
    # 2. Pré-processamento inicial
    df['datetime_segundo_audiencia'] = pd.to_datetime(df['datetime_segundo_audiencia'], errors='coerce')
    df = df.sort_values('datetime_segundo_audiencia')

    # 3. Preparar dados focando em retrancas de conteúdo
    df_filtrado = df[df['Retranca'].notna() & (df['Retranca'] != "Não disponível")].copy()
    
    tipos_a_excluir = ['ABER', 'IMG', 'VINH']
    tipos_excluidos_str = ", ".join(tipos_a_excluir)
    
    tipo_col_name = None
    if 'tipo' in df_filtrado.columns:
        tipo_col_name = 'tipo'
    elif 'Tipo' in df_filtrado.columns:
        tipo_col_name = 'Tipo'
        
    if tipo_col_name:
        print(f"Coluna '{tipo_col_name}' encontrada. Excluindo tipos: {tipos_excluidos_str}.")
        df_retrancas_validas = df_filtrado[~df_filtrado[tipo_col_name].isin(tipos_a_excluir)].copy()
    else:
        print(f"AVISO: Nenhuma coluna 'tipo' ou 'Tipo' foi encontrada. O filtro por tipo de retranca não foi aplicado.")
        df_retrancas_validas = df_filtrado.copy()


    if df_retrancas_validas.empty:
        print("Não há dados de 'Retranca' definidos para esta análise após os filtros.")
    elif 'GLOBO' not in df_retrancas_validas.columns or not pd.api.types.is_numeric_dtype(df_retrancas_validas['GLOBO']):
        print("ERRO: Coluna 'GLOBO' não encontrada ou não é numérica.")
    else:
        print("Identificando instâncias de Retrancas e calculando métricas...")
        
        # 4. Identificar instâncias (blocos) de Retrancas
        df_retrancas_validas['Retranca_Shifted'] = df_retrancas_validas['Retranca'].shift(1)
        df_retrancas_validas['novo_bloco_retranca'] = (df_retrancas_validas['Retranca'] != df_retrancas_validas['Retranca_Shifted'])
        df_retrancas_validas['bloco_id_sequencial'] = df_retrancas_validas['novo_bloco_retranca'].cumsum()

        # 5. Calcular métricas por instância de Retranca
        inicio_blocos_retranca = df_retrancas_validas.groupby('bloco_id_sequencial').agg(Retranca=('Retranca', 'first'),horario_inicio_bloco=('datetime_segundo_audiencia', 'min')).reset_index()
        audiencia_por_bloco = df_retrancas_validas.groupby('bloco_id_sequencial').agg(audiencia_media_bloco=('GLOBO', 'mean'),duracao_bloco_segundos=('GLOBO', 'count')).reset_index()
        df_plot_temporal_barras = pd.merge(inicio_blocos_retranca, audiencia_por_bloco, on='bloco_id_sequencial')
        
        df_plot_temporal_barras = df_plot_temporal_barras.sort_values(by='horario_inicio_bloco')
        
        # 7. Criar label para o eixo X
        df_plot_temporal_barras['Retranca'] = df_plot_temporal_barras['Retranca'].astype(str)
        
        # --- Lógica para criar labels únicos e evitar sobreposição ---
        df_plot_temporal_barras['ocorrencia'] = df_plot_temporal_barras.groupby('Retranca').cumcount() + 1
        df_plot_temporal_barras['total_ocorrencias'] = df_plot_temporal_barras.groupby('Retranca')['Retranca'].transform('count')
        df_plot_temporal_barras['label_eixo_x'] = df_plot_temporal_barras.apply(
            lambda row: f"{row['Retranca']} ({row['ocorrencia']})" if row['total_ocorrencias'] > 1 else row['Retranca'],
            axis=1
        )


        if df_plot_temporal_barras.empty:
            print(f"Nenhuma instância de retranca encontrada após os filtros.")
        else:
            num_instancias = len(df_plot_temporal_barras)
            print(f"Preparando gráfico de BARRAS para {num_instancias} instâncias de Retrancas em ordem de aparição...")

            # O valor da média deve ser o mesmo usado na linha de referência do gráfico.
            media_programa = 8.19 
            
            df_plot_temporal_barras['Performance'] = np.where(
                df_plot_temporal_barras['audiencia_media_bloco'] > media_programa,
                'Acima da média',
                'Abaixo ou na média'
            )

            chart_width = max(800, num_instancias * 35)

            # 8. Criar o Gráfico de Barras Verticais
            fig = px.bar(
                df_plot_temporal_barras,
                x='label_eixo_x',
                y='audiencia_media_bloco',
                title='Audiência por Matérias',
                labels={'label_eixo_x': 'Matérias (Ordem de Aparição)', 
                        'audiencia_media_bloco': 'Audiência Domiciliar (pp)',
                        'Performance': 'Legenda'},
                hover_data={'Retranca': True, 'horario_inicio_bloco': True, 'duracao_bloco_segundos': True,'label_eixo_x': False },
                color='Performance',
                color_discrete_map={
                    'Acima da média': 'mediumseagreen', # Verde
                    'Abaixo ou na média': 'cornflowerblue' # Azul
                },
                width=chart_width
            )
            
            fig.update_traces(
                hovertemplate="<b>Matéria:</b> %{customdata[0]}<br>" + "<b>Início:</b> %{customdata[1]|%H:%M:%S}<br>" + "<b>Audiência Média:</b> %{y:.2f}<br>" + "<b>Duração:</b> %{customdata[2]}s<extra></extra>"
            )
            
            fig.update_traces(
                texttemplate='%{y:.2f}',
                textposition='inside',
                insidetextanchor='middle',
                textfont=dict(
                    color='white',
                    size=12
                ),
                constraintext='inside'
            )

            # 9. Estilizar o Gráfico
            nome_programa = "BDES" 
            
            # ALTERAÇÃO: A data da análise agora é definida manualmente aqui.
            # Basta editar o texto dentro das aspas.
            data_analise = "13/06/2025"
            
            fig.update_layout(
                xaxis_title='Matéria em Ordem de Aparição no Programa',
                yaxis_title='Audiência Domiciliar (pp)',
                title={
                    'text': f"<b>Audiência por Matérias - {nome_programa}</b><br>" +
                            f"<sup>Análise de: {data_analise}  |  Fonte: Kantar - Ibope</sup><br>" +
                            f"<sup><i>Desconsideradas retrancas do tipo: {tipos_excluidos_str}</i></sup>",
                    'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                    'font': {'size': 18}
                },
                xaxis_tickangle=-60,
                showlegend=True
            )
            
            fig.add_hline(
                y=8.19,
                line_width=2, 
                line_dash="dot", 
                line_color="black"
            )
            
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='lines',
                line=dict(color='black', dash='dot', width=2),
                name='Média simples do dia (8.19)',
                showlegend=True
            ))

            fig.update_xaxes(categoryorder='array', categoryarray=df_plot_temporal_barras['label_eixo_x'])

            # 10. Salvar em HTML
            output_html_path_temporal_barras = "audiencia_por_materias_BDES.html"
            fig.write_html(output_html_path_temporal_barras)

            print(f"\nGráfico de Barras de Audiência de Retrancas salvo como: '{output_html_path_temporal_barras}'")
            print("Abra este arquivo HTML em um navegador para interagir com o gráfico.")
            
            # Para exibir diretamente no Jupyter Notebook, descomente a linha abaixo:
            # fig.show()
else:
    print("\nO DataFrame não foi carregado. A visualização não pode ser gerada.")

Arquivo 'audiencia_espelho_merged_segundos.csv' carregado com sucesso!
Coluna 'Tipo' encontrada. Excluindo tipos: ABER, IMG, VINH.
Identificando instâncias de Retrancas e calculando métricas...
Preparando gráfico de BARRAS para 58 instâncias de Retrancas em ordem de aparição...

Gráfico de Barras de Audiência de Retrancas salvo como: 'audiencia_por_materias_BDES.html'
Abra este arquivo HTML em um navegador para interagir com o gráfico.


In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1. Carregar o dataframe detalhado
file_path = "audiencia_espelho_merged_segundos.csv"
df = None 

try:
    df = pd.read_csv(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{file_path}' não encontrado.")
    print("Por favor, verifique o nome e o local do arquivo.")

# Prosseguir somente se o dataframe foi carregado com sucesso
if df is not None:
    # 2. Pré-processamento inicial
    df['datetime_segundo_audiencia'] = pd.to_datetime(df['datetime_segundo_audiencia'], errors='coerce')
    df = df.sort_values('datetime_segundo_audiencia')

    # 3. Preparar dados focando em retrancas de conteúdo
    df_filtrado = df[df['Retranca'].notna() & (df['Retranca'] != "Não disponível")].copy()
    
    tipos_a_excluir = ['ABER', 'IMG', 'VINH']
    tipos_excluidos_str = ", ".join(tipos_a_excluir)
    
    tipo_col_name = None
    if 'tipo' in df_filtrado.columns:
        tipo_col_name = 'tipo'
    elif 'Tipo' in df_filtrado.columns:
        tipo_col_name = 'Tipo'
        
    if tipo_col_name:
        print(f"Coluna '{tipo_col_name}' encontrada. Excluindo tipos: {tipos_excluidos_str}.")
        df_retrancas_validas = df_filtrado[~df_filtrado[tipo_col_name].isin(tipos_a_excluir)].copy()
    else:
        print(f"AVISO: Nenhuma coluna 'tipo' ou 'Tipo' foi encontrada. O filtro por tipo de retranca não foi aplicado.")
        df_retrancas_validas = df_filtrado.copy()


    if df_retrancas_validas.empty:
        print("Não há dados de 'Retranca' definidos para esta análise após os filtros.")
    elif 'GLOBO' not in df_retrancas_validas.columns or not pd.api.types.is_numeric_dtype(df_retrancas_validas['GLOBO']):
        print("ERRO: Coluna 'GLOBO' não encontrada ou não é numérica.")
    else:
        print("Identificando instâncias de Retrancas e calculando métricas...")
        
        # 4. Identificar instâncias (blocos) de Retrancas
        df_retrancas_validas['Retranca_Shifted'] = df_retrancas_validas['Retranca'].shift(1)
        df_retrancas_validas['novo_bloco_retranca'] = (df_retrancas_validas['Retranca'] != df_retrancas_validas['Retranca_Shifted'])
        df_retrancas_validas['bloco_id_sequencial'] = df_retrancas_validas['novo_bloco_retranca'].cumsum()

        # 5. Calcular métricas por instância de Retranca
        inicio_blocos_retranca = df_retrancas_validas.groupby('bloco_id_sequencial').agg(Retranca=('Retranca', 'first'),horario_inicio_bloco=('datetime_segundo_audiencia', 'min')).reset_index()
        audiencia_por_bloco = df_retrancas_validas.groupby('bloco_id_sequencial').agg(audiencia_media_bloco=('GLOBO', 'mean'),duracao_bloco_segundos=('GLOBO', 'count')).reset_index()
        df_plot_temporal_barras = pd.merge(inicio_blocos_retranca, audiencia_por_bloco, on='bloco_id_sequencial')
        
        df_plot_temporal_barras = df_plot_temporal_barras.sort_values(by='horario_inicio_bloco')
        
        # 7. Criar label para o eixo X
        df_plot_temporal_barras['Retranca'] = df_plot_temporal_barras['Retranca'].astype(str)
        
        df_plot_temporal_barras['ocorrencia'] = df_plot_temporal_barras.groupby('Retranca').cumcount() + 1
        df_plot_temporal_barras['total_ocorrencias'] = df_plot_temporal_barras.groupby('Retranca')['Retranca'].transform('count')
        df_plot_temporal_barras['label_eixo_x'] = df_plot_temporal_barras.apply(
            lambda row: f"{row['Retranca']} ({row['ocorrencia']})" if row['total_ocorrencias'] > 1 else row['Retranca'],
            axis=1
        )

        if df_plot_temporal_barras.empty:
            print(f"Nenhuma instância de retranca encontrada após os filtros.")
        else:
            num_instancias = len(df_plot_temporal_barras)
            print(f"Preparando gráfico de BARRAS para {num_instancias} instâncias de Retrancas em ordem de aparição...")

            media_programa = 8.19 
            
            df_plot_temporal_barras['Performance'] = np.where(
                df_plot_temporal_barras['audiencia_media_bloco'] > media_programa,
                'Acima da média',
                'Abaixo ou na média'
            )

            chart_width = max(800, num_instancias * 35)

            # 8. Criar o Gráfico de Barras Verticais
            fig = px.bar(
                df_plot_temporal_barras,
                x='label_eixo_x',
                y='audiencia_media_bloco',
                title='Audiência por Matérias',
                labels={'label_eixo_x': 'Matérias (Ordem de Aparição)', 
                        'audiencia_media_bloco': 'Audiência Domiciliar (pp)',
                        'Performance': 'Legenda'},
                hover_data={'Retranca': True, 'horario_inicio_bloco': True, 'duracao_bloco_segundos': True,'label_eixo_x': False },
                color='Performance',
                color_discrete_map={
                    'Acima da média': 'mediumseagreen', 
                    'Abaixo ou na média': 'cornflowerblue'
                },
                width=chart_width
            )
            
            fig.update_traces(
                hovertemplate="<b>Matéria:</b> %{customdata[0]}<br>" + "<b>Início:</b> %{customdata[1]|%H:%M:%S}<br>" + "<b>Audiência Média:</b> %{y:.2f}<br>" + "<b>Duração:</b> %{customdata[2]}s<extra></extra>"
            )
            
            # --- ALTERAÇÕES APLICADAS AQUI ---
            fig.update_traces(
                texttemplate='%{y:.1f}',      # MUDANÇA 1: Mostra só 1 casa decimal
                textposition='inside',
                insidetextanchor='middle',
                textangle=0,                  # MUDANÇA 2: Força o texto a ficar na horizontal
                textfont=dict(
                    color='white',
                    size=12
                ),
                constraintext='inside'
            )

            # 9. Estilizar o Gráfico
            nome_programa = "BDES" 
            data_analise = "13/06/2025"
            
            fig.update_layout(
                xaxis_title='Matéria em Ordem de Aparição no Programa',
                yaxis_title='Audiência Domiciliar (pp)',
                title={
                    'text': f"<b>Audiência por Matérias - {nome_programa}</b><br>" +
                            f"<sup>Análise de: {data_analise}  |  Fonte: Kantar - Ibope</sup><br>" +
                            f"<sup><i>Desconsideradas retrancas do tipo: {tipos_excluidos_str}</i></sup>",
                    'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                    'font': {'size': 18}
                },
                xaxis_tickangle=-60,
                showlegend=True
            )
            
            fig.add_hline(
                y=8.19,
                line_width=2, 
                line_dash="dot", 
                line_color="black"
            )
            
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='lines',
                line=dict(color='black', dash='dot', width=2),
                name='Média simples do dia (8.19)',
                showlegend=True
            ))

            fig.update_xaxes(categoryorder='array', categoryarray=df_plot_temporal_barras['label_eixo_x'])

            # 10. Salvar em HTML
            output_html_path_temporal_barras = "audiencia_por_materias_BDES_2.html"
            fig.write_html(output_html_path_temporal_barras)

            print(f"\nGráfico de Barras de Audiência de Retrancas salvo como: '{output_html_path_temporal_barras}'")
            print("Abra este arquivo HTML em um navegador para interagir com o gráfico.")
            
            # Para exibir diretamente no Jupyter Notebook, descomente a linha abaixo:
            # fig.show()
else:
    print("\nO DataFrame não foi carregado. A visualização não pode ser gerada.")

Arquivo 'audiencia_espelho_merged_segundos.csv' carregado com sucesso!
Coluna 'Tipo' encontrada. Excluindo tipos: ABER, IMG, VINH.
Identificando instâncias de Retrancas e calculando métricas...
Preparando gráfico de BARRAS para 58 instâncias de Retrancas em ordem de aparição...

Gráfico de Barras de Audiência de Retrancas salvo como: 'audiencia_por_materias_BDES_2.html'
Abra este arquivo HTML em um navegador para interagir com o gráfico.
